In [24]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
55,Lovingly crafted and terribly interesting to w...,positive
781,I have this movie on a collection of inexpensi...,positive
233,I saw this movie with very low expectations. I...,negative
430,I love Jane Austen's stories. I've only read t...,negative
109,this isn't 'Bonnie and Clyde' or 'Thelma and L...,positive


In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aakil\AppData\Roaming\nltk_data...


True

In [8]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\aakil\AppData\Local\Temp\ipykernel_13704\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
55,lovingly crafted terribly interesting watch ga...,positive
781,movie collection inexpensive b movie restored ...,positive
233,saw movie low expectation know lot sure going ...,negative
430,love jane austen story read two p p seen adapt...,negative
109,bonnie clyde thelma louise fine road movie set...,positive


In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [13]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
55,lovingly crafted terribly interesting watch ga...,1
781,movie collection inexpensive b movie restored ...,1
233,saw movie low expectation know lot sure going ...,0
430,love jane austen story read two p p seen adapt...,0
109,bonnie clyde thelma louise fine road movie set...,1


In [14]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/mdakeel/mlops-capstone-project.mlflow')
dagshub.init(repo_owner='mdakeel', repo_name='mlops-capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Accessing as mdakeel

Initialized MLflow to track repo "mdakeel/mlops-capstone-project"

Repository mdakeel/mlops-capstone-project initialized!

2025/09/17 14:00:43 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/22c5aa7294f84adb9492fc19dbd201e8', creation_time=1758097843586, experiment_id='0', last_update_time=1758097843586, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [34]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.3)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")
        # mlflow.sklearn.log_model(model, artifact_path="model")
        
        joblib.dump(model, "../models/model.pkl")
        mlflow.log_artifact("../models/model.pkl")



        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-09-17 14:27:40,329 - INFO - Starting MLflow run...
2025-09-17 14:27:40,843 - INFO - Logging preprocessing parameters...
2025-09-17 14:27:42,277 - INFO - Initializing Logistic Regression model...
2025-09-17 14:27:42,279 - INFO - Fitting the model...
2025-09-17 14:27:42,373 - INFO - Model training complete.
2025-09-17 14:27:42,373 - INFO - Logging model parameters...
2025-09-17 14:27:42,786 - INFO - Making predictions...
2025-09-17 14:27:42,786 - INFO - Calculating evaluation metrics...
2025-09-17 14:27:42,818 - INFO - Logging evaluation metrics...
2025-09-17 14:27:48,826 - INFO - Saving and logging the model...
2025-09-17 14:27:49,958 - INFO - Model training and logging completed in 9.11 seconds.
2025-09-17 14:27:49,964 - INFO - Accuracy: 0.68
2025-09-17 14:27:49,968 - INFO - Precision: 0.6463414634146342
2025-09-17 14:27:49,968 - INFO - Recall: 0.7361111111111112
2025-09-17 14:27:49,975 - INFO - F1 Score: 0.6883116883116883


🏃 View run honorable-sponge-600 at: https://dagshub.com/mdakeel/mlops-capstone-project.mlflow/#/experiments/0/runs/b3dc51645aae4bf5a98e41e7eb3a66bc
🧪 View experiment at: https://dagshub.com/mdakeel/mlops-capstone-project.mlflow/#/experiments/0
